# Extracting explanations from a sparse audio classifier

In [1]:
from audIBle.nn.autoencoders import SpecAE
from audIBle.nn.sparse_classif import SparseClassifier
from audIBle.data.datasets import UrbanSound8k

import torch
import numpy as np
import json
import os

In [2]:
config_ids = ["001", "002", "003"]
seed=42

id_idx = 1
exp_name = f"{config_ids[id_idx]}_sparse_classif_urbasound8k_{seed}"
exp_root = os.path.join(os.environ["EXP_ROOT"], "train/SAE/ae_debug/",exp_name)

with open(os.path.join(exp_root, "config.json"), 'r') as fh:
    cfg = json.load(fh)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

autoencoder = SpecAE(**cfg["model"]["autoencoder"])
ae_ckpt_path = cfg["model"]["ae_ckpt_path"]
if ae_ckpt_path is not None:
    ae_ckpt = torch.load(ae_ckpt_path, map_location=device, weights_only=True)
    autoencoder.load_state_dict(ae_ckpt)

# prepare the sparse classifier
classif_params = cfg["model"]["classifier"]
classif_params["autoencoder"] = autoencoder
model = SparseClassifier(**classif_params)

In [4]:
# classifier checkpoint
classif_ckpt_path = os.path.join(exp_root,"best_model.pth")
ckpt = torch.load(classif_ckpt_path, map_location=device, weights_only=True)
model.load_state_dict(ckpt)

<All keys matched successfully>

In [5]:
data_root = "/lium/corpus/vrac/tmario/sed/urbansound8k/urbansound8k"
fold_test = 10
dataset = UrbanSound8k(csv_path=os.path.join(data_root, "metadata/UrbanSound8K.csv"),
                           audio_dir=os.path.join(data_root, "audio"),
                           sample_rate=cfg["sample_rate"],
                           folds_to_use=[10],)

In [ ]:
def compute_relevance(model, tau, input_data, label, device):
    model.to(device)

    # extract all of the representations
    all_rep = model.forward_return_all(input_data.to(device))
    # pooled sparse representation (used to make the decision)
    Z = all_rep["sparse_latent_pooled"]
    # weights of the linear classification head
    W_c = model.classif_head.weight
    #weights of the SAE decoder (dictionary)
    W_sae_d = model.sae.decoder.weight

    n_classes = W_c.shape[0]

    for c in range(n_classes):
        # select the weights of the classification layer according to class c
        w_class = W_c[c,:]
        r_c = 

In [6]:
model.to(device)
W_c = model.classif_head.weight
W_sae_d = model.sae.decoder.weight

print(W_c.shape)
print(W_sae_d.shape)
for i_data in range(len(dataset)):
    wav, label = dataset[i_data]
    wav = wav.unsqueeze(0)

    all_rep = model.forward_return_all(wav.to(device))

    for k,v in all_rep.items():
        print(f"{k}: {v.shape}")

    


    break

torch.Size([10, 1024])
torch.Size([256, 1024])
spec: torch.Size([1, 1, 513, 345])
spec_hat: torch.Size([1, 1, 513, 345])
ae_hidden: torch.Size([1, 345, 256])
ae_hidden_hat: torch.Size([1, 345, 256])
sparse_latent: torch.Size([1, 345, 1024])
sparse_latent_pooled: torch.Size([1, 1024])
attention_weights: torch.Size([1, 345, 1])
logits: torch.Size([1, 10])
